<a href="https://colab.research.google.com/github/marciorlis/My_First_Colab-Imersao_Alura/blob/main/imersao_IA_Alura_%2B_Google_aula_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
!pip install -U -q google-generativeai

In [54]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [55]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [56]:
#exemplo de embedding
title = "A próxima geração de IA"
sample_text = ("Titulo: A rpóxima geração de IA"
"\n"
"Artigo completo:\n"
"Geminai API& Google AI Studio")

embeddings = genai.embed_content(model="models/embedding-001",
content=sample_text,
                                title=title,
                                task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

{'embedding': [0.034416266, -0.018969303, -0.009016063, -0.034522954, 0.069709755, -0.020435141, -0.0032589955, -0.016353453, 0.029364083, 0.05992827, 0.013304481, 0.048881736, -0.032623738, -0.0059961462, 0.021294752, -0.022884712, 0.0281159, -0.006907488, -0.0328009, 0.00732547, 0.012293295, 0.0015904743, -0.01848225, -0.05244992, -0.021515235, 0.008969908, 0.007996395, -0.018789064, -0.03601859, 0.033396278, -0.0266224, 0.043138795, -0.03399809, 0.013761663, -0.029849669, -0.04784105, -0.04424445, -0.04120767, 0.03170731, 0.004808537, 0.014079928, -0.08375133, -0.017176786, 0.024084544, 0.0031384148, -0.040235203, 0.014748352, 0.02983072, 0.015965246, -0.06660956, 0.035718877, 0.0073662433, 0.07397562, -0.053493347, -0.012087559, 0.01607884, 0.04376014, -0.02372642, 0.018816633, 0.006709658, 0.008803966, -0.00022178006, -0.024986988, 0.0651959, -0.0057554813, -0.041149125, -0.035029072, -0.0066212676, 0.03122617, 0.017073309, 0.018570673, -0.014958288, 0.0761456, -0.06351925, -0.015

In [57]:
DOCUMENT1 = {
    "Título": "Operação do sistema de conterole climatioc",
    "Conteúdo": "O Googlecar tem um sistema de controle climaico que permite ajustar a temperatura e  o fluxo de ar no carro"}
DOCUMENT2 = {
    "Título": "Touchscreen",
    "Conteúdo": "O Googlecar tem uma grande tela sensivel ao toque que fornece a =cesso a uma variedade de recusos"}
DOCUMENT3 = {
    "Título": "Mudanças de marchas",
    "Conteúdo": "O Googlecar tem uma transmissão automatica. Para trocas as marchars basta mover a alaavnca de cambio"}

documents = [DOCUMENT1,DOCUMENT2,DOCUMENT3]

In [58]:
df = pd.DataFrame(documents)
df.columns = ["Titulo","Conteudo"]
df

,Titulo,Conteudo
0,Operação do sistema de conterole climatioc,O Googlecar tem um sistema de controle climaic...
1,Touchscreen,O Googlecar tem uma grande tela sensivel ao to...
2,Mudanças de marchas,O Googlecar tem uma transmissão automatica. Pa...


In [59]:
model = "models/embedding-001"

In [60]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=sample_text,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [61]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Operação do sistema de conterole climatioc,O Googlecar tem um sistema de controle climaic...,"[0.017912373, -0.025128383, -0.043707494, 0.00..."
1,Touchscreen,O Googlecar tem uma grande tela sensivel ao to...,"[0.041792866, -0.046805102, -0.024949746, -0.0..."
2,Mudanças de marchas,O Googlecar tem uma transmissão automatica. Pa...,"[0.038826693, -0.012294431, -0.010151718, -0.0..."


In [65]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                             content=consulta,
                             task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [67]:
consulta = "Como é a tela do carro do Google?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

O Googlecar tem uma grande tela sensivel ao toque que fornece a =cesso a uma variedade de recusos


In [73]:
generation_config = {
    "temperature": 0.5,
    "candidate_count":1,
}

In [84]:
prompt = f"Reescreva esse texto de uma forma mais descontraída. sem adicionar informaçoes que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

O Googlecar tem uma telona irada que é tipo um tablet gigante. Você pode usar ela pra dar um rolê na internet, ouvir musiquinha ou até mesmo pedir uma pizza!
